In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
import time


In [2]:
def printTable(table):
    '''A simple way to display matrices in a more readable format'''
    for i in table:
        print(i)

In [3]:
def evaluate_util(player_row):
    if players[player_row -1][4] == True:
        restOfROI_main_role = combos[(centers - 1, wings, defenders, goalies, utils)][player_row - 1][budget_interval - players[player_row - 1][1]]
        restOfROI_util = combos[(centers, wings, defenders, goalies, utils - 1)][player_row - 1][budget_interval - players[player_row - 1][1]]

In [4]:
def get_optimal_set(players_list, budget, inc, limits):
    
    ## first we divide all working money sums by the increment
    
    players = [[player[0], int(int(player[1])/inc), player[2], player[3]] for player in players_list]
    
    budget = int(budget / inc)
    #for player in players:
    #    player[1] = int(int(player[1]) / inc)

        
    nPlayers = len(players) 
    
    combos = {}
    combos_traceback = {}
    
    for centers in range(limits["C"] + 1):
        for wings in range(limits["W"] + 1):
            for defenders in range(limits["D"] + 1):
                for goalies in range(limits["G"] + 1):
                    combos[(centers, wings, defenders, goalies)] = [[-1 for i in range(budget + 1)] for i in range(nPlayers + 1)]
                    combos_traceback[(centers, wings, defenders, goalies)] = [[None for i in range(budget + 1)] for i in range(nPlayers + 1)]
                
                
    ## First, fill in the base case for 0 forwards, defenders, and goalies
    combos[(0,0,0)] = [[-0 for i in range(budget + 1)] for i in range(nPlayers + 1)]
    
    ## Now we iterate through the combinations of forwards, defenders, and goalies
    for centers in range(limits["C"] + 1):
        for wings in range(limits["W"] + 1):
            for defenders in range(limits["D"] + 1):
                for goalies in range(limits["G"] + 1):
                
                
                    roster_limits = {"C":centers,"W": wings ,"D":defenders ,"G": goalies}
                    table = combos[(centers, wings, defenders, goalies)]
                    traceback = combos_traceback[(centers, wings, defenders, goalies)]
                
                    ## Filling in the base case for 0 players to choose from
                    for i in range(budget + 1):
                        table[0][i] = 0
                    
                    ## Now we incrememnt through each player
                    for player_row in range(1, nPlayers + 1):
                    
                        ## For each player, we increment through our budget
                        for budget_interval in range(budget + 1):
                        
                            ##For each acceptable player, we see if we can afford the player and if we have roster space
                            player_position = players[player_row -1][3]
                            if roster_limits[player_position] > 0 and budget_interval >= players[player_row - 1][1]:
                    
                    
                                ## We calculate the points
                                points = players[player_row - 1][2]
                        
                                ## We look through previous dictionarys to find the rest of the ROI
                                if player_position == "C":
                                    restOfROI = combos[(centers - 1, wings, defenders, goalies)][player_row - 1][budget_interval - players[player_row - 1][1]]
                                elif player_position == "W":
                                    restOfROI = combos[(centers, wings -1 , defenders, goalies)][player_row - 1][budget_interval - players[player_row - 1][1]]
                                elif player_position == "D":
                                    restOfROI = combos[(centers, wings, defenders - 1, goalies)][player_row - 1][budget_interval - players[player_row - 1][1]]
                                elif player_position == "G":
                                    restOfROI = combos[(centers, wings, defenders, goalies - 1)][player_row - 1][budget_interval - players[player_row - 1][1]]
                                
                                        
                                ## We compare it for the optimal value if we dont sign the player
                                noSign = table[player_row - 1][budget_interval]
                
                                ## if signing the player generates more profit than not signing the player, then we add him to the table
                                if points + restOfROI > noSign:
                                    table[player_row][budget_interval] = points + restOfROI
                                    traceback[player_row][budget_interval] = True
                                
                                else:
                                    table[player_row][budget_interval] = noSign
                                    traceback[player_row][budget_interval] = False
                                
                            ## if we cant afford the player or have roster room, we treat it as a noSign
                            else:
                                table[player_row][budget_interval] = table[player_row - 1][budget_interval]
                                traceback[player_row][budget_interval] = False
                            
    ## To return the optimized investment portfolio, we create an empty return list
    ## and initialize our item marker value, and a money iteration value
    ret_list = []
    val = nPlayers
    i = -1
    centers = limits["C"]
    wings = limits["W"]
    defenders = limits["D"]
    goalies = limits["G"]
    
    ## we loop through our table until our value (signifying the current item) is at 0
    while val > 0:
        
        ## If our traceback table tells us to buy the investment
        if combos_traceback[(centers, wings, defenders, goalies )][val][i]:
            
            ## We record the player position
            player_position = players[val - 1][3]
            
            ## We add our investment number, cost, and ROI to our output list
            ret_list.append((players[val - 1][0], players[val - 1][1] * inc, players[val - 1][2], players[val - 1][3]))
                            
            ## We increment our money iteration value backwards by the amount that we "spent" on the investment
            ## And increment our item marker value back by 1
            ## and incrememnt back the roster space
            i -= players[val - 1][1]
            val -= 1
            
            if player_position == "C":
                centers -= 1
            if player_position == "W":
                wings -= 1    
            if player_position == "D":
                defenders -= 1
            elif player_position == "G":
                goalies -= 1

        
        ## If our traceback table tells us to not buy the investment,
        ## we simply increment our item marker value back by 1
        else:
            val -= 1
    
    
    return ret_list#, combos, combos_traceback

In [5]:
players_df = pd.read_csv("DKSalaries_2_25.csv")
players_df["Position"] = players_df["Roster Position"].str[0]
players_df = players_df.drop(columns=["Name + ID","ID","Game Info","TeamAbbrev", "Roster Position"])
players_df.head()

,Position,Name,Salary,AvgPointsPerGame
0,C,Nathan MacKinnon,9000,14.94
1,W,David Pastrnak,8800,20.52
2,G,Tuukka Rask,8200,12.19
3,G,Jaroslav Halak,8200,14.10
4,G,Callum Booth,8000,0.00


In [6]:
data_for_algo = players_df[["Name", "Salary", "AvgPointsPerGame", "Position"]].values.tolist()
data_for_algo[0]

['Nathan MacKinnon', 9000, 14.94, 'C']

In [7]:
limits = {"C": 2,"W": 3 ,"D" : 2,"G": 1}
results = get_optimal_set(data_for_algo, 50000, 100, limits)

In [8]:
results

[('Arthur Kaliyev', 2500, 11.3, 'W'),
 ('Urho Vaakanainen', 3100, 10.8, 'D'),
 ('Mats Zuccarello', 4700, 14.32, 'W'),
 ('Devon Toews', 5600, 11.85, 'D'),
 ('Calvin Petersen', 7200, 16.52, 'G'),
 ('Patrice Bergeron', 7900, 15.16, 'C'),
 ('David Pastrnak', 8800, 20.52, 'W'),
 ('Nathan MacKinnon', 9000, 14.94, 'C')]

In [9]:
cap = 0
points = 0
for player in results:
    cap += player[1]
    points += player[2]
    
print(cap, points)

48800 115.41


In [110]:
players = data_for_algo
taken_by_other = []
selected = []
remaining_players = [player for player in players if player not in taken_by_other or player in selected]

In [111]:
remaining_players[0]

['Nathan MacKinnon', 9000, 14.94, 'C']

In [112]:
def get_player_info(players, target):
    for player in players:
        if player[0] == target:
            return(player)

In [113]:
def draft_from_middle(players, already_selected, players_taken_by_other, budget, inc, limits):
    
    for selected_player in already_selected:
        player_info = get_player_info(players, selected_player)
        budget = budget - player_info[1]
        limits[player_info[3]] -= 1
    
    
    remaining_players = [player for player in players if player[0] not in players_taken_by_other]    
        
    remaining_players = [player for player in remaining_players if player[0] not in already_selected]    
    
    remaining_draft = get_optimal_set(remaining_players, budget, inc, limits)
    
    for already_selected_player in already_selected:
        remaining_draft += [get_player_info(players, already_selected_player)]
    
    return remaining_draft
    

In [114]:
players = players_df[["Name", "Salary", "AvgPointsPerGame", "Position"]].values.tolist()

already_selected = ['Nathan MacKinnon']
players_taken_by_other = ['David Pastrnak']
limits = {"C": 2,"W": 3 ,"D" : 2,"G": 1}


mid_draft = draft_from_middle(players, already_selected, players_taken_by_other, 50000, 100, limits)

In [115]:
mid_draft

[('Arthur Kaliyev', 2500, 11.3, 'W'),
 ('Mats Zuccarello', 4700, 14.32, 'W'),
 ('Devon Toews', 5600, 11.85, 'D'),
 ('Jakob Chychrun', 6300, 11.72, 'D'),
 ('Artemi Panarin', 6400, 13.46, 'W'),
 ('Calvin Petersen', 7200, 16.52, 'G'),
 ('Patrice Bergeron', 7900, 15.16, 'C'),
 ['Nathan MacKinnon', 9000, 14.94, 'C']]

In [159]:
## If you get one extra center
start_time = time.time()
limits_extra_center = {"C": 3,"W": 3 ,"D" : 2,"G": 1}
results_extra_center = get_optimal_set(data_for_algo, 50000, 100, limits_extra_center)
print("--- %s seconds ---" % (time.time() - start_time))

## if you get one extra Wing
start_time = time.time()
limits_extra_wing = {"C": 2,"W": 4 ,"D" : 2,"G": 1}
results_extra_wing = get_optimal_set(data_for_algo, 50000, 100, limits_extra_wing)
print("--- %s seconds ---" % (time.time() - start_time))

## if you get one extra Defender
start_time = time.time()
limits_extra_defender ={"C": 2,"W": 3 ,"D" : 3,"G": 1}
results_extra_defender = get_optimal_set(data_for_algo, 50000, 100, limits_extra_defender)
print("--- %s seconds ---" % (time.time() - start_time))

--- 9.069542407989502 seconds ---
--- 8.352576971054077 seconds ---
--- 8.951764583587646 seconds ---


In [168]:
results_extra_center

[('Arthur Kaliyev', 2500, 11.3, 'W'),
 ('Urho Vaakanainen', 3100, 10.8, 'D'),
 ('Ryan Strome', 4400, 8.26, 'C'),
 ('Mats Zuccarello', 4700, 14.32, 'W'),
 ('Devon Toews', 5600, 11.85, 'D'),
 ('Anze Kopitar', 5700, 11.37, 'C'),
 ('Calvin Petersen', 7200, 16.52, 'G'),
 ('Patrice Bergeron', 7900, 15.16, 'C'),
 ('David Pastrnak', 8800, 20.52, 'W')]

In [167]:
cap = 0
points = 0
for player in results_extra_wing:
    cap += player[1]
    points += player[2]
    
print(cap, points)

49500 122.19
